In [1]:
import sys
import os
import numpy as np
import tensorflow as tf
import pennylane as qml
import pandas as pd
from matplotlib import pyplot as plt

from quantum_neural_network import qnode_entangling, qnode_strong_entangling
from stat_functions import quantitative_analysis, get_mean_left_right_error_interval
from experiments_main import plot_history, plot_prediction_versus_observed, carregar_tabela
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MinMaxScaler



## Importing Data

In [2]:
prev = 1

train = "data/train150_mucuri.txt"
X_all,y_all = carregar_tabela(train, prev)

n_features = X_all.shape[1]
n_instances = X_all.shape[0]
print(f"There are {n_features} features and {n_instances} instances in the train data")

test = "data/prev150_mucuri.txt"
X_test,y_test = carregar_tabela(test,prev)

n_features = X_test.shape[1]
n_instances = X_test.shape[0]
print(f"There are {n_features} features and {n_instances} instances in the test data")

There are 9 features and 549 instances in the train data
There are 9 features and 193 instances in the test data


In [3]:
X_all.head()

,Dia,Mês,Ano,Hora,Velocidade,Direção,Temperatura,Umidade,Pressão
0,30,11,2015,14,13.012139,75.105481,27.516129,72.930636,1020.422601
1,30,11,2015,15,12.726087,68.334332,27.238095,75.212121,1020.394348
2,30,11,2015,16,12.081111,64.457865,27.105263,75.741379,1020.508333
3,30,11,2015,17,11.647222,53.842100,26.305556,75.302632,1020.611000
4,30,11,2015,18,11.064444,53.945279,25.464286,76.592593,1020.866500


In [4]:
y_all[:5]

array([[12.72608696],
       [12.08111113],
       [11.64722224],
       [11.06444444],
       [10.32444445]])

## Scaling Data

In [5]:
scaler_x = MinMaxScaler(feature_range=(-1, 1))
X_all_scaled = scaler_x.fit_transform(X_all)

X_test_scaled = scaler_x.transform(X_test)

## Spliting Train, Validation and Test sets

In [6]:
train_ratio = 0.8
X_train, X_val, y_train, y_val = train_test_split(X_all_scaled, y_all, test_size=1 - train_ratio)

print("Len(Train):",len(X_train))
print("Len(Val):"  ,len(X_val))
print("Len(Test):" ,len(X_test_scaled))

Len(Train): 439
Len(Val): 110
Len(Test): 193


## Quantum Neural Network

In [7]:
n_qubits = n_features
n_layers = 1
print(f"Circuit size: {n_qubits} qubits")

Circuit size: 9 qubits


In [8]:
def create_quantum_model(n_layers, n_qubits):
    print(f"Training with depth {n_layers}")
    weight_shapes = {"weights": (n_layers,n_qubits,3)}

    input_layer = tf.keras.layers.Input(shape=(n_qubits,))
    q_layer = qml.qnn.KerasLayer(qnode_strong_entangling, weight_shapes, output_dim=n_qubits)
    activation=tf.keras.layers.Activation(tf.keras.activations.relu)
    output_layer = tf.keras.layers.Dense(prev,kernel_initializer='normal')

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

    model = tf.keras.models.Sequential([input_layer, q_layer, activation, output_layer])
    model.compile(loss=['mse'], optimizer=optimizer, metrics=['mae'])

    return model


In [9]:
model = create_quantum_model(n_layers, n_qubits)
model.summary()

Training with depth 1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 9)                 27        
                                                                 
 activation (Activation)     (None, 9)                 0         
                                                                 
 dense (Dense)               (None, 1)                 10        
                                                                 
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
re=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='min', min_lr=0.00001)
history_model = model.fit(X_train, y_train
                        , epochs=50, batch_size=32
                        , callbacks=[re]
                        , verbose=0
                        , validation_data=(X_val, y_val))

## Plotting Loss

In [ ]:
plot_history(history_model, n_layers)

In [ ]:
list_y_pred = []

    #################
    ### Loss Plot ###
    #################

    ##################
    ### Prediction ###
    ##################
    y_pred = model.predict(X_test_scaled,verbose=0)
    y_pred_normal = scaler_y.inverse_transform(y_pred)
    list_y_pred.append(y_pred_normal)
    mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal = get_mean_left_right_error_interval(model, scaler_y, X_val, y_val, y_test, y_pred_normal)
    plot_prediction_versus_observed(n_layers, y_test, y_pred_normal, mean_error_normal)
    print("\n#########\n")
